In [41]:
import numpy as np
import matplotlib.pyplot as plt

from numpy.polynomial import legendre as legendre_np
from scipy.special    import legendre as legendre_sc
from scipy.integrate  import quad, dblquad
from scipy            import linalg   as la

import cupy as cp

# print(plt.style.available)
plt.style.use("seaborn-whitegrid")

In [5]:
### degree
order = 5

### get roots
roots, weights = legendre_np.leggauss(order)

print(np.r_[[roots, weights]])

[[-0.90617985 -0.53846931  0.          0.53846931  0.90617985]
 [ 0.23692689  0.47862867  0.56888889  0.47862867  0.23692689]]


In [6]:
fun = lambda x: 3.0 * x**2 + 1
np.sum([w * fun(x) for w, x in zip(weights, roots)])

4.000000000000002

In [7]:
# integral and estimate error
a, b     = -1, 1
ing, err = quad(fun, a, b)
print(ing)

3.999999999999999


In [9]:
fun = lambda x, y: x * y
print(fun(1, 2))
print(fun(2, 2))

2
4


In [11]:
res = 0
for wx, x in zip(weights, roots):
    for wy, y in zip(weights, roots):
        res += (wx * wy * fun(x, y))
print(res)

6.938893903907228e-18


In [20]:
fun = lambda x, y: x**2 * y**2
res = 0

for wx, x in zip(weights, roots):
    for wy, y in zip(weights, roots):
        res += (wx * wy * fun(x, y))
        
print(res)

0.4444444444444449


In [21]:
ing, err = dblquad(fun, -1, 1, lambda x: -1, lambda x: 1)
print(ing)

0.44444444444444436


In [13]:
4/9

0.4444444444444444

In [15]:
fun = lambda x, y: np.exp(x * y)
res = 0

for wx, x in zip(weights, roots):
    for wy, y in zip(weights, roots):
        res += (wx * wy * fun(x, y))
        
print(res)

4.22900350120645


In [19]:
ing, err = dblquad(fun, -1, 1, lambda x: -1, lambda x: 1)
print(ing)

4.229003501502914


In [28]:
xa, xb = -1, 5
ya, yb = -1, 1

xh, xc = (xb-xa) / 2, (xb+xa) / 2
yh, yc = (yb-ya) / 2, (yb+ya) / 2

xnodes = roots * xh + xc
ynodes = roots * yh + yc

In [29]:
fun = lambda x, y: np.exp(x * y)
res = 0

for wx, x in zip(weights, xnodes):
    for wy, y in zip(weights, ynodes):
        res += (wx * wy * fun(x, y))

res = res * xh * yh        
print(res)

42.29530901039896


In [30]:
ing, err = dblquad(fun, xa, xb, lambda x: ya, lambda x: yb)
print(ing)

42.30092540214591


In [113]:
xa, xb = -1, 5
ya, yb = -1, 1

xh, xc = (xb-xa) / 2, (xb+xa) / 2
yh, yc = (yb-ya) / 2, (yb+ya) / 2

xnodes = roots * xh + xc
ynodes = roots * yh + yc

vfun = np.vectorize(fun)

In [114]:
%%time
X, Y = np.meshgrid(xnodes, ynodes)

W1, W2 = np.meshgrid(weights, weights)
W      = np.multiply(W1, W2)

ing = np.multiply(vfun(X, Y), W).sum() * xh * yh
print(ing)

42.29530901039896
CPU times: user 1.15 ms, sys: 466 µs, total: 1.62 ms
Wall time: 1.26 ms


In [110]:
xa, xb = -1, 5
ya, yb = -1, 1

xh, xc = (xb-xa) / 2, (xb+xa) / 2
yh, yc = (yb-ya) / 2, (yb+ya) / 2

xnodes = roots * xh + xc
ynodes = roots * yh + yc

xc = cp.array(xnodes)
yc = cp.array(ynodes)
wc = cp.array(weights)

In [111]:
@cp.fuse()
def cfun(x, y):
    return cp.exp(x * y)

In [112]:
%%time
X, Y = cp.meshgrid(xc, yc)

W1, W2 = cp.meshgrid(wc, wc)
W      = cp.multiply(W1, W2)

ing = cp.multiply(cfun(X, Y), W).sum() * xh * yh
print(ing)

42.29530901039896
CPU times: user 9 ms, sys: 80 µs, total: 9.08 ms
Wall time: 6.85 ms


In [109]:
@cp.fuse()
def cfun(x, y):
    return cp.exp(x * y)

cfun(X, Y)

array([[1.91769903e+00, 7.05739152e-01, 1.63268419e-01, 3.77711460e-02,
        1.39002920e-02],
       [1.47242616e+00, 8.12945647e-01, 3.40636750e-01, 1.42732046e-01,
        7.88042200e-02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00],
       [6.79151204e-01, 1.23009454e+00, 2.93567855e+00, 7.00613514e+00,
        1.26896757e+01],
       [5.21458259e-01, 1.41695412e+00, 6.12488323e+00, 2.64752359e+01,
        7.19409347e+01]])

In [108]:
cfun = cp.ElementwiseKernel(
    'float64 x, float64 y',
    'float64 z',
    'z = exp(x * y)',
    'cfun')

cfun(X, Y)

array([[1.91769903e+00, 7.05739152e-01, 1.63268419e-01, 3.77711460e-02,
        1.39002920e-02],
       [1.47242616e+00, 8.12945647e-01, 3.40636750e-01, 1.42732046e-01,
        7.88042200e-02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00],
       [6.79151204e-01, 1.23009454e+00, 2.93567855e+00, 7.00613514e+00,
        1.26896757e+01],
       [5.21458259e-01, 1.41695412e+00, 6.12488323e+00, 2.64752359e+01,
        7.19409347e+01]])